## Association Learning

##### using mlxtend library of python, pip install mlxtend
understand the metrics
http://aimotion.blogspot.in/2013/01/machine-learning-and-data-mining.html 

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

##### reading our data

In [2]:
df = pd.read_excel('./data/OnlineRetailReduced.xlsx')
print(df.head())

KeyboardInterrupt: 

### cleaning up data


In [3]:
df['Description'] = df['Description'].str.strip()

#convert full column into str
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

#drop the rows with no invoice number
#modify same object with inplace
#columns to consider while dropping is subset
print("Original size", df.shape)
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
print("After dropping empty invoices", df.shape)

#dropping all invoices with C, just to maintain proper data 
credit_validator = df['InvoiceNo'].str.startswith('C')
df = df[~credit_validator]
print("After dropping credits", df.shape, "\n")

print(df.head())

Original size (541909, 8)
After dropping empty invoices (541909, 8)
After dropping credits (532621, 8) 

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


#### if you dont know Groupby Function Refer to the Pandas_GroupbyExample Notebook

In [4]:
#doing only for france to reduce size and show results
int1 = (df[df['Country'] == "France"]
          .groupby(['InvoiceNo', 'Description']))

#view how the data is looking
# for name, group in int1:
#     print(name, '\n', group, '\n')

#we need only quantity & its sum, here the sum will be the quantity of individual item bought
int2 = int1['Quantity'].sum()
#print(int2)

#convert column description to row
# output should be a matrix..., with value as the quantity
# [invoiceno item1 item2 item3...itemn,
#  invoiceno item1 item2 item3...itemn]
# unstack will do that for us. 
# convert resulting Nans to 0 with fillna
int3 = int2.unstack().fillna(0)
#print(int3)

#get to original panda data frame with index 0, 1...
int4 = int3.reset_index()
#print(int4)

#lets set the index as the invoiceno instead of 0,1,2..n
basket = int4.set_index('InvoiceNo')
print(basket)

Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536370                          0.0                         0.0   
536852                          0.0                         0.0   
536974                          0.0                         0.0   
537065                          0.0                         0.0   
537463                          0.0                         0.0   
537468                         24.0                         0.0   
537693                          0.0                         0.0   
537897                          0.0                         0.0   
537967                          0.0                         0.0   
538008                          0.0                         0.0   
538093                          0.0                         0.0   
538196                          0.0                         0.0   
539050                          0.0                         0.

### find the frequent sets and apply association rules

In [5]:
# convert anything greater than 1 or 0
encode_units = lambda x: 1 if x>0 else 0
basket_sets = basket.applymap(encode_units)

#basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [6]:
#now find out frequeny sets of items
#min_support = 5%
#these itemsets come at least 3% of all transactions in the database
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
print(frequent_itemsets.head())
print(frequent_itemsets.tail())

#self exercises with pandas
#find all the itemsets with length more than 2

    support                       itemsets
0  0.071429  [4 TRADITIONAL SPINNING TOPS]
1  0.096939   [ALARM CLOCK BAKELIKE GREEN]
2  0.102041    [ALARM CLOCK BAKELIKE PINK]
3  0.094388     [ALARM CLOCK BAKELIKE RED]
4  0.068878   [ASSORTED COLOUR MINI CASES]
      support                                           itemsets
190  0.102041  [POSTAGE, SET/6 RED SPOTTY PAPER CUPS, SET/6 R...
191  0.099490  [SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...
192  0.056122  [ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
193  0.053571  [PLASTERS IN TIN CIRCUS PARADE, PLASTERS IN TI...
194  0.081633  [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...


In [163]:
#now apply association rules over the frequent items
#frequent itemsets should have support and itemsets colums
#Minimal threshold for the evaluation metric to decide whether a candidate rule is of interest.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(POSTAGE),(4 TRADITIONAL SPINNING TOPS),0.765306,0.071429,0.056122,0.073333,1.026667,0.001458,1.002055
1,(4 TRADITIONAL SPINNING TOPS),(POSTAGE),0.071429,0.765306,0.056122,0.785714,1.026667,0.001458,1.095238
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
3,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
4,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
6,(ALARM CLOCK BAKELIKE GREEN),(POSTAGE),0.096939,0.765306,0.084184,0.868421,1.134737,0.009996,1.783673
7,(POSTAGE),(ALARM CLOCK BAKELIKE GREEN),0.765306,0.096939,0.084184,0.110000,1.134737,0.009996,1.014676
8,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061
9,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135


In [164]:
input_item = 'ALARM CLOCK BAKELIKE RED'
input_item1 = 'SET/6 RED SPOTTY PAPER PLATES'

rules['new'] = rules['antecedants'].apply(lambda x: input_item in list(x))
rules[rules['new']]

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,new
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,True
8,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061,True
13,(ALARM CLOCK BAKELIKE RED),(POSTAGE),0.094388,0.765306,0.086735,0.918919,1.200721,0.014499,2.894558,True
15,(ALARM CLOCK BAKELIKE RED),(ROUND SNACK BOXES SET OF4 WOODLAND),0.094388,0.158163,0.051020,0.540541,3.417611,0.036092,1.832233,True
176,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE PINK),0.079082,0.102041,0.063776,0.806452,7.903226,0.055706,4.639456,True
178,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE GREEN),0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194,True
180,(ALARM CLOCK BAKELIKE RED),"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",0.094388,0.073980,0.063776,0.675676,9.133271,0.056793,2.855230,True
189,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(POSTAGE),0.079082,0.765306,0.071429,0.903226,1.180215,0.010907,2.425170,True
190,"(POSTAGE, ALARM CLOCK BAKELIKE RED)",(ALARM CLOCK BAKELIKE GREEN),0.086735,0.096939,0.071429,0.823529,8.495356,0.063021,5.117347,True
193,(ALARM CLOCK BAKELIKE RED),"(ALARM CLOCK BAKELIKE GREEN, POSTAGE)",0.094388,0.084184,0.071429,0.756757,8.989353,0.063483,3.765023,True
